In [1]:
from src import clear_subs, Love_numbers_for_options_for_models_for_asymptotic_mu_ratios

"""
clear_subs()
anelasticity_description_ids = Love_numbers_for_options_for_models_for_asymptotic_mu_ratios(
    asymptotic_mu_ratios=[0.2],
    elasticity_model_names=["PREM"],
    long_term_anelasticity_model_names=[
        "Burgers_asthenosphere__elastic_lithosphere",
        "Peltier_upper_mantle__elastic_lithosphere",
    ],
    short_term_anelasticity_model_names=["Benjamin_Q_PREM"],
)
"""

Description: PREM___Burgers_asthenosphere__elastic_lithosphere___Benjamin_Q_PREM/asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0:
----Run: long_term_anelasticity__short_term_anelasticity__bounded_attenuation_functions: Done.
----Run: __short_term_anelasticity__bounded_attenuation_functions: Done.
----Run: long_term_anelasticity____: Done.
----Run: __short_term_anelasticity__: Done.
----Run: long_term_anelasticity__short_term_anelasticity__: Done.
Description: PREM___Peltier_upper_mantle__elastic_lithosphere___Benjamin_Q_PREM/asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0:
----Run: long_term_anelasticity__short_term_anelasticity__bounded_attenuation_functions: Done.
----Run: long_term_anelasticity____: Done.
----Run: long_term_anelasticity__short_term_anelasticity__: Done.


In [2]:
from src import get_single_float_Love_number

get_single_float_Love_number(
    anelasticity_desciption_id="PREM___Burgers_asthenosphere__elastic_lithosphere___Benjamin_Q_PREM"
    + "/"
    + "asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0"
)[0][0]

(1.336086707571451-0.01700261053809833j)

In [1]:
from src import BoundaryCondition, plot_Love_numbers_for_options_for_descriptions_per_type

plot_Love_numbers_for_options_for_descriptions_per_type(
    anelasticity_description_ids=[
        "PREM___Peltier_upper_mantle__elastic_lithosphere___Benjamin_Q_PREM"
        + "/"
        + "asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0",
        "PREM___Burgers_asthenosphere__elastic_lithosphere___Benjamin_Q_PREM"
        + "/"
        + "asymptotic_mu_ratio__0.2__0.2__0.2__0.2__1.0",
    ],
    boundary_conditions=[BoundaryCondition.load, BoundaryCondition.shear, BoundaryCondition.potential],
)

/home/mrousselet/Code/SNREI/src/scripts/plot/Love_numbers.py:143: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  plt.clf()


<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

...

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>